USING ARIMA MODEL TO PREDICT TIME SERIES DATA FORTHE NEXT 7 DAYS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [2]:
#loading the dataset
df = pd.read_csv('IndianWeatherRepository.csv')

In [3]:
#remove redundant attributes
col_to_remove=['country','latitude','longitude','timezone','last_updated_epoch','temperature_celsius','wind_mph','wind_direction','pressure_in','precip_in','feels_like_celsius','feels_like_fahrenheit','visibility_miles','gust_mph','uv_index','gust_kph','sunrise','sunset','moonrise','moonset','moon_phase','moon_illumination','location_name','air_quality_Carbon_Monoxide', 'air_quality_Ozone','air_quality_Nitrogen_dioxide','air_quality_Sulphur_dioxide','air_quality_PM2.5','air_quality_PM10','air_quality_us-epa-index','air_quality_gb-defra-index']
df = df.drop(columns=col_to_remove)
df

,region,last_updated,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
0,Madhya Pradesh,2023-08-29 10:45,81.5,Partly cloudy,20.5,281,1008.0,0.00,67,26,10.0
1,Madhya Pradesh,2023-08-29 10:45,81.5,Sunny,15.5,287,1008.0,0.00,70,19,10.0
2,Madhya Pradesh,2023-08-29 10:45,79.3,Partly cloudy,18.4,317,1009.0,0.00,70,51,10.0
3,Madhya Pradesh,2023-08-29 10:45,78.1,Cloudy,16.9,297,1009.0,0.00,76,65,10.0
4,Madhya Pradesh,2023-08-29 10:45,81.0,Cloudy,16.2,274,1009.0,0.00,74,82,10.0
...,...,...,...,...,...,...,...,...,...,...,...
53067,Uttar Pradesh,2023-12-08 01:15,59.0,Mist,3.6,10,1016.0,0.00,82,0,1.6
53068,Mizoram,2023-12-08 01:15,67.8,Patchy rain possible,3.6,82,1013.0,0.06,100,100,10.0
53069,Tamil Nadu,2023-12-08 01:15,74.7,Partly cloudy,4.0,71,1011.0,0.00,93,34,10.0
53070,Tamil Nadu,2023-12-08 01:15,73.1,Partly cloudy,6.5,282,1011.0,0.00,92,55,10.0


In [4]:
#extract the rows whose region name is Andaman and Nicobar Islands
condition=df['region']=='Andaman and Nicobar Islands'
df = df[condition]
df

,region,last_updated,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
187,Andaman and Nicobar Islands,2023-08-29 10:45,82.0,Light rain shower,36.7,259,1009.0,0.30,80,70,10.0
730,Andaman and Nicobar Islands,2023-08-30 08:45,81.1,Light rain shower,22.0,269,1010.0,1.60,79,75,10.0
1280,Andaman and Nicobar Islands,2023-08-31 05:15,82.2,Cloudy,31.7,242,1009.0,0.00,83,69,10.0
1830,Andaman and Nicobar Islands,2023-09-01 05:15,81.7,Moderate or heavy rain shower,40.0,243,1010.0,3.90,82,83,7.0
2374,Andaman and Nicobar Islands,2023-09-02 05:15,80.6,Moderate or heavy rain shower,33.1,237,1009.0,3.30,83,90,7.0
...,...,...,...,...,...,...,...,...,...,...,...
50517,Andaman and Nicobar Islands,2023-12-04 02:00,81.0,Patchy rain possible,18.7,85,1008.0,0.21,81,70,9.0
51066,Andaman and Nicobar Islands,2023-12-05 01:00,80.4,Patchy rain possible,7.6,129,1008.0,1.16,81,81,9.0
51612,Andaman and Nicobar Islands,2023-12-06 01:15,80.6,Partly cloudy,12.6,134,1009.0,0.00,84,28,10.0
52159,Andaman and Nicobar Islands,2023-12-07 01:15,80.3,Light rain shower,8.6,160,1010.0,0.97,80,83,10.0


In [5]:
#different weather names used
distinct_values = df['condition_text'].unique().tolist()
print(distinct_values)
print("There are "+str(len(distinct_values))+" distinct weather types in the dataset")

['Light rain shower', 'Cloudy', 'Moderate or heavy rain shower', 'Torrential rain shower', 'Thundery outbreaks possible', 'Partly cloudy', 'Patchy rain possible', 'Clear', 'Light drizzle', 'Patchy light drizzle', 'Patchy light rain', 'Heavy rain']
There are 12 distinct weather types in the dataset


In [6]:
#percentage of missing values
missing_values=df.isnull()
miss_percent=(missing_values.sum()/len(df))*100
missing_data=pd.DataFrame({'missing percent': miss_percent})
print(missing_data)

                        missing percent
region                              0.0
last_updated                        0.0
temperature_fahrenheit              0.0
condition_text                      0.0
wind_kph                            0.0
wind_degree                         0.0
pressure_mb                         0.0
precip_mm                           0.0
humidity                            0.0
cloud                               0.0
visibility_km                       0.0


In [7]:
#identify and remove duplicated rows
duplicated_rows=df.duplicated(keep=False)
print(duplicated_rows)
duplicate_data=df[duplicated_rows]
print(duplicate_data)
df=df.drop_duplicates(keep='first')
print('number of rows: ',df.shape[0])

187      False
730      False
1280     False
1830     False
2374     False
         ...  
50517    False
51066    False
51612    False
52159    False
52709    False
Length: 97, dtype: bool
Empty DataFrame
Columns: [region, last_updated, temperature_fahrenheit, condition_text, wind_kph, wind_degree, pressure_mb, precip_mm, humidity, cloud, visibility_km]
Index: []
number of rows:  97


In [8]:
#data type of each row
data_types = df.dtypes
print(data_types)

region                     object
last_updated               object
temperature_fahrenheit    float64
condition_text             object
wind_kph                  float64
wind_degree                 int64
pressure_mb               float64
precip_mm                 float64
humidity                    int64
cloud                       int64
visibility_km             float64
dtype: object


In [9]:
'''my_column = df['last_updated']
modified_column = my_column.apply(lambda x: x[:-5])
df['last_updated'] = modified_column
df
'''

"my_column = df['last_updated']\nmodified_column = my_column.apply(lambda x: x[:-5])\ndf['last_updated'] = modified_column\ndf\n"

In [10]:
#convert the int64 data to float64
df['humidity'] = df['humidity'].astype('float64')
df['wind_degree'] = df['wind_degree'].astype('float64')
df['cloud'] = df['cloud'].astype('float64')

data_types = df.dtypes
print(data_types)

region                     object
last_updated               object
temperature_fahrenheit    float64
condition_text             object
wind_kph                  float64
wind_degree               float64
pressure_mb               float64
precip_mm                 float64
humidity                  float64
cloud                     float64
visibility_km             float64
dtype: object


ENCODING CONDITION TEXT

In [11]:

from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the column and transform the values
df['condition_text'] = encoder.fit_transform(df['condition_text'])
df['condition_text'] = df['condition_text'].astype(int)

# Access the encoded classes
encoded_classes = encoder.classes_
#print("Encoded Classes:", encoded_classes)

# Decode the encoded values to get the original labels
decoded_labels = encoder.inverse_transform(df['condition_text'])
#print("Decoded Labels:", decoded_labels)

ARIMA FOR NEXT 7 DAYS

In [12]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Set the date column as the index
df['last_updated'] = pd.to_datetime(df['last_updated'])
df.set_index('last_updated', inplace=True)

# Select the parameters for prediction
parameters = ['temperature_fahrenheit', 'wind_kph', 'wind_degree', 'pressure_mb',
              'precip_mm', 'humidity', 'cloud', 'visibility_km']

# Create an empty DataFrame to store predictions
predictions_df = pd.DataFrame()
# Iterate over each parameter
for param in parameters:
    # Split the data into training and testing sets
    train_data = df[:-7][param]
    test_data = df[-7:][param]

    # Fit an ARIMA model
    model = ARIMA(train_data, order=(5, 2, 2))  # Adjust the order as needed
    model_fit = model.fit()

    # Make predictions for the next 7 days
    predictions = model_fit.forecast(steps=7)

    # Print the predictions
    print(f"Predictions for {param}:")
    print(predictions)
    print()
    predictions_df[param] = predictions

# Print the predictions DataFrame
print(predictions_df)

# Calculate accuracy metrics
mae = mean_absolute_error(test_data, predictions)
mse = mean_squared_error(test_data, predictions)
rmse = mean_squared_error(test_data, predictions, squared=False)

# Print the accuracy metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros a

Predictions for temperature_fahrenheit:
90    80.845736
91    81.169735
92    80.842672
93    80.681078
94    80.864197
95    80.619125
96    80.599540
Name: predicted_mean, dtype: float64



c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so wil

Predictions for wind_kph:
90    27.421531
91    25.884471
92    24.335316
93    23.585810
94    24.897752
95    24.629687
96    25.273281
Name: predicted_mean, dtype: float64

Predictions for wind_degree:
90    79.602833
91    75.382151
92    76.110527
93    70.780766
94    72.239034
95    66.786314
96    68.015838
Name: predicted_mean, dtype: float64



c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so wil

...

c:\Users\HP\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date 

Predictions for precip_mm:
90    1.833574
91    1.284997
92    0.439997
93    0.541208
94    0.267646
95    0.503549
96    0.501144
Name: predicted_mean, dtype: float64



c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so wil

Predictions for humidity:
90    86.072551
91    85.886855
92    86.335615
93    87.226210
94    86.367269
95    86.913419
96    86.822906
Name: predicted_mean, dtype: float64

Predictions for cloud:
90    83.802234
91    75.945575
92    81.940880
93    80.769588
94    82.470835
95    85.656381
96    83.585313
Name: predicted_mean, dtype: float64

Predictions for visibility_km:
90    8.781131
91    8.883221
92    9.148555
93    8.855121
94    8.670249
95    8.572362
96    8.932835
Name: predicted_mean, dtype: float64

    temperature_fahrenheit   wind_kph  wind_degree  pressure_mb  precip_mm  \
90               80.845736  27.421531    79.602833  1008.191335   1.833574   
91               81.169735  25.884471    75.382151  1008.058666   1.284997   
92               80.842672  24.335316    76.110527  1008.055768   0.439997   
93               80.681078  23.585810    70.780766  1008.010026   0.541208   
94               80.864197  24.897752    72.239034  1008.053256   0.267646   
95       

c:\Users\HP\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\HP\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'


NORMALISE BEFORE CLASSIFICATION

In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Select only the numerical columns
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the numerical columns and transform the values
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
df


,region,temperature_fahrenheit,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,visibility_km
last_updated,,,,,,,,,,
2023-08-29 10:45:00,Andaman and Nicobar Islands,0.350427,4,0.835859,0.736994,0.777778,0.022438,0.454545,0.673913,1.000
2023-08-30 08:45:00,Andaman and Nicobar Islands,0.273504,4,0.464646,0.765896,0.888889,0.119671,0.409091,0.728261,1.000
2023-08-31 05:15:00,Andaman and Nicobar Islands,0.367521,1,0.709596,0.687861,0.777778,0.000000,0.590909,0.663043,1.000
2023-09-01 05:15:00,Andaman and Nicobar Islands,0.324786,5,0.919192,0.690751,0.888889,0.291698,0.545455,0.815217,0.625
2023-09-02 05:15:00,Andaman and Nicobar Islands,0.230769,5,0.744949,0.673410,0.777778,0.246821,0.590909,0.891304,0.625
...,...,...,...,...,...,...,...,...,...,...
2023-12-04 02:00:00,Andaman and Nicobar Islands,0.264957,9,0.381313,0.234104,0.666667,0.015707,0.500000,0.673913,0.875
2023-12-05 01:00:00,Andaman and Nicobar Islands,0.213675,9,0.101010,0.361272,0.666667,0.086761,0.500000,0.793478,0.875
2023-12-06 01:15:00,Andaman and Nicobar Islands,0.230769,6,0.227273,0.375723,0.777778,0.000000,0.636364,0.217391,1.000


DECISION TREE CLASSIFIER

In [14]:
# we are now going to approach the solution using DECISION TREE CLASSIFIER
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Load the dataset
X = df.iloc[:, [1,3,4,5,6,7,8,9]].values
y = df.iloc[:, 2].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create a decision tree classifier
clf = tree.DecisionTreeClassifier(random_state=18)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

Accuracy: 85.0 %


In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Select only the numerical columns
numerical_columns = predictions_df.select_dtypes(include=['float64', 'int64']).columns

# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the numerical columns and transform the values
predictions_df[numerical_columns] = scaler.fit_transform(predictions_df[numerical_columns])
pred=clf.predict(predictions_df)
#print(pred)

# Decode the encoded values to get the original labels
decoded_labels = encoder.inverse_transform(pred)

# Print the decoded labels
print("Decoded Labels:", decoded_labels)

Decoded Labels: ['Torrential rain shower' 'Patchy light drizzle' 'Light rain shower'
 'Patchy light drizzle' 'Partly cloudy' 'Patchy light drizzle'
 'Patchy light drizzle']


In [16]:
last_7_rows = df.tail(7)
#print(last_7_rows['condition_text'])
# Decode the encoded values to get the original labels
decoded_labels = encoder.inverse_transform(last_7_rows['condition_text'])

# Print the decoded labels
print("Decoded Labels:", decoded_labels)


Decoded Labels: ['Heavy rain' 'Patchy rain possible' 'Patchy rain possible'
 'Patchy rain possible' 'Partly cloudy' 'Light rain shower' 'Clear']
